In [7]:
import numpy as np
N=10

mat = np.zeros((N,N))
for i in range(N-1):
    mat[i, i+1] = 1
    mat[i+1, i] = 1

print(mat)
print(np.linalg.eig(mat)[0])

def normalize(sigma):
    dim = sigma.shape[0]
    # Impose PD and mean of eigenvalues equal 1

    # Forces all diagonal elements to be the same
    # That's the easiest way to ensure all ev > 0...
    sigma -= np.diag(np.diag(sigma))
    sigma += np.diag([dim for _ in range(dim)])

    sigma *= dim / np.sum(np.linalg.eig(sigma)[0])
    assert np.all(np.linalg.eig(sigma)[0] > 0.)
    assert np.abs(np.sum(np.linalg.eig(sigma)[0]) - dim) < 0.05

    return sigma

mat2 = normalize(mat)
print(mat2)
print(np.linalg.eig(mat2)[0])

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
[-1.91898595 -1.68250707 -1.30972147 -0.83083003 -0.28462968  0.28462968
  1.91898595  1.68250707  1.30972147  0.83083003]
[[1.  0.1 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.1 1.  0.1 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.1 1.  0.1 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.1 1.  0.1 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.1 1.  0.1 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.1 1.  0.1 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.1 1.  0.1 0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.1 1.  0.1 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.1 1.  0.1]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.1 1. ]]
[0.80810141 0.83174929 0.86902785 0.916917   0.97153703 1.02846297
 1.19189859 1.16825071 1.13097215 1.083083  ]
